In [ ]:
pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 41.8 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/My Drive/pegasus_data.csv")


In [ ]:
texts = df["cleaned_comment"].tolist()
labels = df["class"].tolist()

In [ ]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load data (assuming texts and labels are ready)
texts = df["cleaned_comment"].tolist()
labels = df["class"].tolist()

# Compute embeddings for retrieval
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(texts)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

# Retrieval function
def retrieve_similar(query, top_k=3):
    query_embedding = embedder.encode([query])
    _, indices = index.search(query_embedding, top_k)
    return [(texts[i], labels[i]) for i in indices[0]]

# Split data
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.3, random_state=42)

# Load and configure model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define custom Dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Prepare training data
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
train_labels = torch.tensor(y_train)
train_dataset = CustomDataset(train_encodings, train_labels)

# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    run_name="distilbert-rac-experiment",  # Unique run name
    num_train_epochs=3,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # Disable W&B logging
)
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset)
trainer.train()

# Classification with retrieval
def classify_with_rag(text):
    similar_samples = retrieve_similar(text)
    context = " ".join([s[0] for s in similar_samples])
    input_text = f"{context} [SEP] {text}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.logits.argmax().item()

# Evaluation
def evaluate_model():
    model.eval()
    predictions = []
    for text in X_test:
        predictions.append(classify_with_rag(text))
    accuracy = accuracy_score(y_test, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    return predictions

# Run evaluation
# predictions = evaluate_model()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.705600
20,0.697000
30,0.694100
40,0.683700
50,0.678400
60,0.678200
70,0.657700
80,0.673000
90,0.622300
100,0.665700


In [ ]:
# Run evaluation
predictions = evaluate_model()

Accuracy: 0.9747


In [3]:
# Define the save directory in Google Drive
save_dir = "/content/drive/My Drive/saved_model"

# Create the directory if it doesn't exist
import os
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Save the trained DistilBERT model and tokenizer
model.save_pretrained(f"{save_dir}/distilbert_model")
tokenizer.save_pretrained(f"{save_dir}/distilbert_model")

# Save the FAISS index
faiss.write_index(index, f"{save_dir}/faiss_index.bin")

# Save the original texts and labels (for retrieval reference)
import pickle
with open(f"{save_dir}/texts_and_labels.pkl", "wb") as f:
    pickle.dump({"texts": texts, "labels": labels}, f)

print(f"Model, tokenizer, FAISS index, and texts/labels saved successfully to {save_dir}!")

Model, tokenizer, FAISS index, and texts/labels saved successfully to /content/drive/My Drive/saved_model!
